# torchtext 
——这是` PyTorch `官方的 `自然语言处理`（NLP）工具包，和` torchvision `在计算机视觉领域的地位类似。

## 1. torchtext 的作用

`PyTorch `自身是一个 `深度学习框架`，不包含太多 NLP 任务专用工具。
`torchtext `提供了常见的 文本处理功能，方便我们快速搭建 NLP 模型，主要包括：

- 1.数据处理:

    - 处理原始文本数据（分词、转小写、去除标点）
    
    - 将文本转为数字索引（token → id）
    
    - 构建 Vocab（词表）
    
    - 批处理 & 填充（padding、masking）

- 2.数据集:

    - 提供常用 NLP 数据集（IMDB、AG_NEWS、SST2、WikiText2 等）
    
    - 内置下载 & 预处理

- 3.词向量

    - 提供预训练的词向量（GloVe、FastText、CharNGram）
    
    - 可以直接加载并用于模型初始化

- 4.与 PyTorch 无缝衔接

    - 提供 `Dataset`、`DataLoader`、`Iterator`
    
    - 和 `torch.nn`、`torchtext.vocab`、`torchtext.transforms` 等模块配合使用

---
## 2. 主要模块结构

`torchtext` 主要分为以下几个部分：

### （1）torchtext.data（旧 API，`已逐渐被弃用`）

- Field：定义如何处理数据（分词、数值化、padding 等）

- Dataset：保存文本数据

- Iterator：按 batch 迭代

⚠️ 注意：新版本推荐用 `torchtext.datasets` 和 `torchtext.transforms`，但一些教程还在用旧 API。

---
### （2）`torchtext.datasets`

提供开箱即用的 NLP 数据集，自动下载和处理。

In [ ]:
from torchtext.datasets import AG_NEWS

train_iter = AG_NEWS(split='train')
test_iter = AG_NEWS(split='test')


`train_iter` 就是一个迭代器，里面包含 `(label, text)`。

---
### （3）torchtext.vocab

提供词表管理 & 预训练词向量

In [ ]:
from torchtext.vocab import GloVe

glove = GloVe(name="6B", dim=100)  # 加载 GloVe 100维词向量
print(glove["computer"])           # 直接取向量


也可以用` build_vocab_from_iterator() `来自己构建词表。

---
### （4）torchtext.transforms

- 提供现代化的数据处理流水线（tokenize → numericalize → pad）

In [ ]:
from torchtext.transforms import BasicEnglishNormalize, VocabTransform, ToTensor

# 基础预处理
text_transform = BasicEnglishNormalize()
tokens = text_transform("Hello, World!")
print(tokens)  # ['hello', 'world']


## BasicEnglishNormalize 
是一个非常实用的 `文本预处理工具`，主要用于 `英文文本的标准化处理`，让输入的文本更干净、统一，便于后续分词和建模。

### 作用

`BasicEnglishNormalize` 会对输入的字符串做以下操作：

- 1.全部转小写

"Hello WORLD!" → "hello world !"

- 2.去掉不必要的符号

把 "、's、n't 等处理为标准化形式

- 3.处理标点符号

把标点和单词分开，例如：
"hello,world!" → "hello , world !"

- 4.标准化常见缩写

"I'm" → "i 'm"

"don't" → "do n't"

⚠️ 注意：它 `不会分词`，只是把文本转换成 `标准化后`的` token `列表。

---
### 和 get_tokenizer("basic_english") 的关系

`BasicEnglishNormalize()` 是一个 `标准化器`，返回 `词元列表`

`get_tokenizer("basic_english")` 内部其实就是调用了` BasicEnglishNormalize`，效果差不多，但写法更简洁

示例对比：

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.transforms import BasicEnglishNormalize

normalizer = BasicEnglishNormalize()
tokenizer = get_tokenizer("basic_english")

text = "I'm using TorchText, It's great!"

print("normalizer:", normalizer(text))
print("tokenizer :", tokenizer(text))

输出：
normalizer: ['i', "'m", 'using', 'torchtext', ',', 'it', "'s", 'great', '!']
tokenizer : ['i', "'m", 'using', 'torchtext', ',', 'it', "'s", 'great', '!']


### ✅ 总结：

`BasicEnglishNormalize` = `基本英文规范化工具`

- 功能：小写化、标点分离、缩写拆分

- 常用于：数据清洗、词表构建、保持词向量一致性

- 如果`只是想快速分词`，可以直接用` get_tokenizer("basic_english")`。

---

## VocabTransform 
是一个 `文本数值化工具`，主要作用就是：

👉 把词（token）转为对应的整数索引（id），也就是所谓的 `numericalization`。
这一步通常在 构建词表` Vocab `之后使用。

## 1. 作用

- 输入：一个 token 序列（如 ["hello", "world"]）

- 输出：对应的索引序列（如 [42, 1337]）

用于把文本转成 可以送进深度学习模型的数字形式。

⚠️ `VocabTransform `依赖一个` Vocab `对象（词表），它只是一个`“映射层”`。

## 2. 使用方法
### （1）构建词表 Vocab

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data_iter):
    for text in data_iter:
        yield text.split()

# 假设我们有一些样本
samples = ["hello world", "hello pytorch"]

# 构建词表
vocab = build_vocab_from_iterator(yield_tokens(samples), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

此时 vocab 的内容可能是：

{'<unk>':0, 'hello':1, 'world':2, 'pytorch':3}


### （2）创建 VocabTransform

In [ ]:
from torchtext.transforms import VocabTransform

# 创建 VocabTransform
vocab_transform = VocabTransform(vocab)


### （3）使用 VocabTransform

In [ ]:
tokens = ["hello", "world", "torch"]
ids = vocab_transform(tokens)

print("tokens:", tokens)
print("ids   :", ids)

输出：

tokens: ['hello', 'world', 'torch']
ids   : [1, 2, 0]

👉 "torch" 不在词表里，变成 <unk> 的索引 0。

## 3. 常见场景

文本预处理流水线`（Pipeline）`
通常和 `BasicEnglishNormalize` 或 `get_tokenizer` 搭配，用来把原始文本转成` id `序列。

示例：

In [ ]:
from torchtext.transforms import Sequential, BasicEnglishNormalize, ToTensor

text_pipeline = Sequential(
    BasicEnglishNormalize(),   # 文本标准化
    vocab_transform,           # token → id
    ToTensor(padding_value=0)  # 转成张量，自动 padding
)

text = "Hello TorchText world!"
print(text_pipeline(text))


## 4. 总结

- 功能：`VocabTransform` = `token` → `id `的映射工具

- 依赖：需要先构建好` Vocab`

- 优势：能和其他 `transform `组合成流水线（`Sequential`）

典型流程：

原始文本 → `BasicEnglishNormalize` → `Token 列表` → `VocabTransform` → `索引序列` → `ToTensor` → `模型输入`

---

---

## 3. 简单示例：文本分类（AG_NEWS）

In [ ]:
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader

# 1. 加载数据
train_iter = AG_NEWS(split='train')

# 2. 分词器
tokenizer = get_tokenizer("basic_english")

# 3. 构建词表
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# 4. 数值化函数
def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(label - 1)  # 标签从0开始
        tokens = tokenizer(text)
        ids = vocab(tokens)
        texts.append(torch.tensor(ids, dtype=torch.int64))
    labels = torch.tensor(labels, dtype=torch.int64)
    texts = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return texts, labels

# 5. DataLoader
train_iter, test_iter = AG_NEWS()
train_loader = DataLoader(train_iter, batch_size=8, shuffle=True, collate_fn=collate_batch)

# 6. 取一个 batch
for X, y in train_loader:
    print("文本 batch:", X.shape)
    print("标签 batch:", y.shape)
    break


## 4. 总结

✅ torchtext 提供的功能：

- 数据集：IMDB、AG_NEWS、WikiText2 等

- 数据处理：分词、数值化、padding、masking

- 词向量：GloVe、FastText、CharNGram

- 与 PyTorch 无缝集成：Dataset、DataLoader、Iterator

⚠️ 版本差异：

- 旧版 API（torchtext.data）：有 Field、Dataset、Iterator

- 新版 API（torchtext.datasets & torchtext.transforms）：更模块化，推荐使用